## Загрузка данных

In [ ]:
main_dir = '/content/metric_learning_classifire'
zip_file = '/Fruits_Vegetables.zip'
data_dir = '/data'
train_dir = '/train'
test_dir = '/test'
save_path_ = '/checkpoint/'

# wandb
path_wandb = 'volkov-timm/image_recognition/dataset:v0'
filename = 'Fruits_Vegetables.zip'
unpack_path = main_dir + '/data'

# train:
pretrained = True
#model = models.resnet18(pretrained=pretrained).to(device)
lr = 3e-4
gamma = 0.5
weight = None
size_average = None

epochs = 1
batch_size = 32

In [ ]:
# IMPORTS -q
!pip install -q wandb 
!pip install -q pytorch-metric-learning[with-hooks]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 4.0 MB/s eta 0:00:00


In [ ]:
# download data
import shutil
#import config
import wandb
from pathlib import Path
import os


def data_extract(filename: str, extract_dir:str) -> None:
    """
    Скачивание датасета
    Распаковка архива в необходимую директорию

    filename - путь до датасета /.zip
    extract_dir - директория для извлечения
    """
    if Path(extract_dir).is_dir() == False:
      os.makedirs(extract_dir)
      
      run = wandb.init()
      artifact = run.use_artifact(path_wandb, type='dataset')
      artifact.download(extract_dir)

      shutil.unpack_archive(filename, extract_dir)
      os.remove(filename)


In [ ]:
data_extract(
      main_dir + data_dir + zip_file,
      main_dir + data_dir
    )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact dataset:v0, 2032.05MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:22.1


In [ ]:
# create df

import glob
import cv2
import pandas as pd
from sklearn import preprocessing
from pathlib import Path

def pivot_df_data(
    data_path: str
) -> pd.DataFrame:
    '''
    Create data frame train and test data
    path|str(label)|train or test|int(label)
    '''
    
    p = Path(data_path)

    labels = [x.name for x in p.iterdir()]
    le = preprocessing.LabelEncoder()
    targets = le.fit_transform(labels)

    # Датафрейм для работы с данными
    for img_ in glob.glob((data_path + '/*/*/*.p*')):
      png_img = cv2.imread(img_)
        
      # converting png_to_jpg
      cv2.imwrite(
          img_[:-4] + '.jpg',
          png_img,
          [int(cv2.IMWRITE_JPEG_QUALITY), 100]
          )

    df = pd.DataFrame({'image': glob.glob((data_path + '/*/*/*.j*'))})
    df['label'] = df['image'].apply(lambda p: p.split('/')[-2])
    df['split'] = df['image'].apply(lambda p: p.split('/')[-3])
    df['int_label'] = le.fit_transform(df['label'])

    return df

In [ ]:
df = pivot_df_data(main_dir + data_dir)

In [ ]:
num_class = df[df['split'] == 'train']['label'].value_counts().shape[0]

In [ ]:
%matplotlib inline
import logging

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
import umap
from cycler import cycler
from PIL import Image
from torchvision import datasets, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2 as ToTensor

import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

# посмотреть как работает
logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

INFO:root:VERSION 2.0.1


### Create embedding

# Transform

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size = 224

train_transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                     (0.2023, 0.1994, 0.2010)),
            ])
val_transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), 
                             (0.2023, 0.1994, 0.2010)),
    ])

batch_size = 32

# DataSet

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
from typing import Tuple
from torchvision.transforms import transforms
import torch
import pandas as pd

class CastomDataset(Dataset):
    
    def __init__(self, df: pd.DataFrame, mode, transforms=None):
        self.df = df[df.split == mode]
        self.transforms = transforms
    
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        row = self.df.iloc[index, :]
        # принудительно конвертируем в RGB из RGBA
        image = Image.open(row['image']).convert('RGB')

        label = row['int_label']

        if self.transforms:
            image = self.transforms(image)

        return image, label
    
    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = CastomDataset(
    df, 
    mode='train',
    transforms=train_transform
)

val_dataset = CastomDataset(
    df, 
    mode='validation', 
    transforms=val_transform
)

test_dataset = CastomDataset(
    df, 
    mode='test', 
    transforms=val_transform
)

## Simple NN

In [ ]:
class MLP(nn.Module):
    """
    Модель для создания эмбеддингов
    layer_sizes: размер поносвязной сети
    """

    def __init__(self, layer_sizes: int, final_relu: bool=False):
        super().__init__()
        layer_list = []
        layer_sizes = [int(x) for x in layer_sizes]
        num_layers = len(layer_sizes) - 1
        final_relu_layer = num_layers if final_relu else num_layers - 1
        for i in range(len(layer_sizes) - 1):
            input_size = layer_sizes[i]
            curr_size = layer_sizes[i + 1]
            if i < final_relu_layer:
                layer_list.append(nn.ReLU(inplace=False))
            layer_list.append(nn.Linear(input_size, curr_size))
        self.net = nn.Sequential(*layer_list)
        self.last_linear = self.net[-1]

    def forward(self, x):
        return self.net(x)

Initialize models, optimizers and image transforms

In [ ]:
#from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set trunk model and replace the softmax layer with an identity function
trunk = torchvision.models.resnet18(pretrained=True)
trunk_output_size = trunk.fc.in_features
trunk.fc = nn.Identity()
trunk = torch.nn.DataParallel(trunk.to(device))

# Set embedder model. This takes in the output of the trunk and outputs 64 dimensional embeddings
embedder = torch.nn.DataParallel(MLP([trunk_output_size, 128]).to(device))

# Set the classifier. The classifier will take the embeddings and output a 50 dimensional vector.
# (Our training set will consist of the first 50 classes of the CIFAR100 dataset.)
# We'll specify the classification loss further down in the code.
classifier = torch.nn.DataParallel(MLP([128, num_class])).to(device)

# Set optimizers
trunk_optimizer = torch.optim.Adam(trunk.parameters(), lr=0.00001, weight_decay=0.0001)
embedder_optimizer = torch.optim.Adam(
    embedder.parameters(), lr=0.0001, weight_decay=0.0001
)
classifier_optimizer = torch.optim.Adam(
    classifier.parameters(), lr=0.0001, weight_decay=0.0001
)

#accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
'''
from pytorch_metric_learning.utils import accuracy_calculator

class YourCalculator(accuracy_calculator.AccuracyCalculator):
    def mean_average_precision_at_18(self, knn_labels, query_labels, **kwargs):
        return accuracy_calculator.mean_average_precision_at_r(knn_labels, query_labels[:, None], 6)
        '''

'\nfrom pytorch_metric_learning.utils import accuracy_calculator\n\nclass YourCalculator(accuracy_calculator.AccuracyCalculator):\n    def mean_average_precision_at_18(self, knn_labels, query_labels, **kwargs):\n        return accuracy_calculator.mean_average_precision_at_r(knn_labels, query_labels[:, None], 6)\n        '

In [ ]:
df.columns

Index(['image', 'label', 'split', 'int_label'], dtype='object')

In [ ]:
train_labels = df[df['split'] == 'train']['int_label']

In [ ]:
wandb.init(project='metric-learning')

# Set the loss function
loss = losses.CosFaceLoss(num_classes=num_class, embedding_size=128).to(device)
#loss = losses.SubCenterArcFaceLoss(num_classes=num_class, embedding_size=128).to(device)
#loss = losses.TripletMarginLoss(margin=0.1)

# Set the classification loss:
classification_loss = torch.nn.CrossEntropyLoss()

# Set the mining function
#miner = miners.MultiSimilarityMiner(epsilon=0.1)

# Set the dataloader sampler
#sampler = samplers.MPerClassSampler(
#    train_labels, m=4, length_before_new_iter=len(train_dataset)
#)

# Set other training parameters

num_epochs = 5



In [ ]:
models = {"trunk": trunk, "embedder": embedder, "classifier": classifier}
optimizers = {
    "trunk_optimizer": trunk_optimizer,
    "embedder_optimizer": embedder_optimizer,
    "classifier_optimizer": classifier_optimizer,
}
loss_funcs = {"metric_loss": loss, "classifier_loss": classification_loss}
#mining_funcs = {"tuple_miner": miner}

# We can specify loss weights if we want to. This is optional
#loss_weights = {"metric_loss": 1, "classifier_loss": 0.5}

In [ ]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=2,
        pin_memory=True,
    )

val_loader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

In [ ]:
import torch.nn as nn
from typing import Callable
import tqdm


def train_one_epoch(
    models: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizers: torch.optim.Optimizer,
    
) -> Tuple[float, float]:

    trunk = models["trunk"].train()
    embedder = models["embedder"].train()
    classifier = models["classifier"].train()

    trunk_optimizer = torch.optim.AdamW(trunk.parameters(), lr=lr)
    embedder_optimizer = torch.optim.AdamW(
        embedder.parameters(), lr=lr)
    classifier_optimizer = torch.optim.AdamW(
        classifier.parameters(), lr=lr)

    loss_f = losses.CosFaceLoss(num_classes=num_class, embedding_size=128).to(device)
    #loss = losses.SubCenterArcFaceLoss(num_classes=num_class, embedding_size=128).to(device)
    #loss = losses.TripletMarginLoss(margin=0.1)

    # Set the classification loss:
    classification_loss = torch.nn.CrossEntropyLoss()
    

    train_running_loss = 0.0
    train_running_correct = 0
    total_epoch_steps = int(len(dataloader.dataset)/dataloader.batch_size)

    for _, batch in tqdm(enumerate(dataloader), total=total_epoch_steps):
        data, labels = batch
        data, labels = data.to(device), labels.to(device)

        output_trunk = trunk(data)
        output_embedding = embedder(output_trunk)
        output_classifier = classifier(output_embedding)
        
        loss = loss_f(output_embedding, labels) \
               + 0.5 * classification_loss(output_classifier, labels)
        train_running_loss += loss.item()

        _, preds = torch.max(output_classifier.data, 1) # можем не использовать softmax, а просто взять .max
        train_running_correct += (preds == labels).sum().item()

        trunk_optimizer.zero_grad()
        embedder_optimizer.zero_grad()
        classifier_optimizer.zero_grad()
        
        loss.backward()

        trunk_optimizer.step()
        embedder_optimizer.step()
        classifier_optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = train_running_correct/len(dataloader.dataset)    
    return train_loss, train_accuracy

In [ ]:
len(df[df['split'] == 'validation']['label'].value_counts())

36

In [ ]:
from itertools import count
import torch
import torch.nn as nn
from torch.utils.data import Dataset

def eval_epoch(models: nn.Sequential, val_loader: Dataset) -> tuple['float', 'float']:
    """
    Валидация одной эпохи
    """
   
    trunk = models["trunk"].eval()
    embedder = models["embedder"].eval()
    classifier = models["classifier"].eval()

    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    last_label = None
    flag = True
    k = 1
    count_true = 0
    list_p_k = []
    list_ap_k = []
    

    for data, labels in val_loader:
        data = data.to(device)
        labels = labels.to(device)

        
        if last_label != labels:
          k = 1
          count_true = 0
          flag = True
          list_p_k = []
          last_label = labels
        elif k < 5: #min_count_val
          k += 1
        else:
          list_ap_k.append(sum(list_p_k) / len(list_p_k)) 
          flag = False


        with torch.set_grad_enabled(False):

            output_trunk = trunk(data)
            output_embedding = embedder(output_trunk)
            output_classifier = classifier(output_embedding)

            loss = loss_funcs['metric_loss'](output_embedding, labels) \
               + 0.5 * loss_funcs['classifier_loss'](output_classifier, labels)

            preds = torch.argmax(output_classifier, 1)

            if flag:
              if preds == labels:
                count_true += int(preds == labels)
                list_p_k.append(count_true / k)
              else:
                list_p_k.append(0)

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == labels)
        processed_size += data.size(0)



        
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    map_k = sum(list_ap_k) / len(list_ap_k)
    
    return val_loss, val_acc, map_k

In [ ]:
from tqdm.notebook import tqdm

def train(
        train_loader,
        val_loader,
        models: nn.Sequential,
        epochs: int,
        #lr: float,
        #gamma: float
) -> list[tuple[float, float, float, float]]:
    """
    Обучение модели, логирование гиперпараметров,
    сохранение наилучшей модели
    """

    wandb.init(project='HW_metric-learning')

    loss = losses.CosFaceLoss(num_classes=num_class, embedding_size=128).to(device)
    classification_loss = torch.nn.CrossEntropyLoss()

    loss_funcs = {"metric_loss": loss, "classifier_loss": classification_loss}


    history = []  # сохраняем данные о loss и accuracy для train и val

    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f} \
    map_k {map_k:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            train_loss, train_acc = train_one_epoch(models,
                                              train_loader,
                                              loss_funcs,
                                              )

            print("loss", train_loss)

            val_loss, val_acc, map_k = eval_epoch(models, val_loader)

            #scheduler.step()

            history.append((train_loss, train_acc, val_loss, val_acc, map_k))

            wandb.log({"train_loss": train_loss,
                       "val_loss": val_loss,
                       "epoch": epoch,
                       "train_acc": train_acc,
                       "val_acc": val_acc,
                       "map_k": map_k
                       })

            pbar_outer.update(1)
            tqdm.write(log_template.format(
                ep=epoch + 1,
                t_loss=train_loss,
                v_loss=val_loss,
                t_acc=train_acc,
                v_acc=val_acc,
                map_k=map_k
                ))

            
            

    return history

In [ ]:
train(train_loader,
        val_loader,
        models,
        15)

epoch:   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

loss 0.6055178256411301

Epoch 001 train_loss: 0.6055     val_loss 36.6192 train_acc 0.2421 val_acc 0.6174     map_k 0.5521


  0%|          | 0/95 [00:00<?, ?it/s]

loss 0.4968244543201045

Epoch 002 train_loss: 0.4968     val_loss 36.0312 train_acc 0.5286 val_acc 0.7826     map_k 0.7158


...

  0%|          | 0/95 [00:00<?, ?it/s]

loss 0.340910998222075

Epoch 009 train_loss: 0.3409     val_loss 34.2307 train_acc 0.8194 val_acc 0.8580     map_k 0.8206


  0%|          | 0/95 [00:00<?, ?it/s]

loss 0.3180199118821245

Epoch 010 train_loss: 0.3180     val_loss 35.0060 train_acc 0.8609 val_acc 0.9217     map_k 0.9046


  0%|          | 0/95 [00:00<?, ?it/s]

In [ ]:
wandb.finish()